In [ ]:
import sys
sys.path.append("../")
import warnings
import pickle
from pathlib import Path
from wsee.utils import utils
from wsee.data import pipeline
import pandas as pd

In [ ]:
warnings.filterwarnings(action='once')
pd.set_option('display.max_colwidth', None)

DATA_DIR = '/Users/phuc/data/daystream_corpus'  # replace path to corpus
WITH_ABSTAINS = True

In [ ]:
loaded_data = pipeline.load_data(DATA_DIR)
sd_train = loaded_data['train']
sd_dev = loaded_data['dev']
sd_test = loaded_data['test']

daystream = loaded_data['daystream']

In [ ]:
merged_event_trigger_examples = pd.read_json(DATA_DIR + '/save_daystreamv6_triggers.jsonl', lines=True, encoding='utf8')
if WITH_ABSTAINS:
    merged_event_role_examples = pd.read_json(DATA_DIR + '/save_daystreamv6_roles_with_abstains.jsonl', lines=True, encoding='utf8')
else:
    merged_event_role_examples = pd.read_json(DATA_DIR + '/save_daystreamv6_roles.jsonl', lines=True, encoding='utf8')
merged_examples: pd.DataFrame = utils.get_deep_copy(daystream)

In [ ]:
for idx, row in merged_examples.iterrows():
    row['event_triggers'] = []
    row['event_roles'] = []
if 'id' in merged_examples:
    merged_examples.set_index('id', inplace=True)

triggers = merged_event_trigger_examples.set_index('id')[['event_triggers']]
roles = merged_event_role_examples.set_index('id')[['event_roles']]

In [ ]:
merged_examples.update(triggers)
merged_examples.update(roles)
print(len(merged_examples))
len(merged_examples[merged_examples['event_triggers'].map(len) > 0])
merged_examples.reset_index(level=0, inplace=True)
# Removes rows with no events
merged_examples = merged_examples[merged_examples['event_triggers'].map(lambda d: len(d)) > 0]

In [ ]:
from wsee.utils.utils import has_events

In [ ]:
examples_with_events = merged_examples.apply(
        lambda document: utils.has_events(document, include_negatives=False), axis=1)

In [ ]:
sum(list(examples_with_events))

In [ ]:
if WITH_ABSTAINS:
    merged_examples.to_json(DATA_DIR + '/daystream_snorkeled_with_abstains.jsonl', orient='records', lines=True, force_ascii=False)
else:
    merged_examples.to_json(DATA_DIR + '/daystream_snorkeled.jsonl', orient='records', lines=True, force_ascii=False)

In [ ]:
merged_examples[['text', 'entities', 'event_triggers', 'event_roles']]